<a href="https://colab.research.google.com/github/kittimaxz/Study/blob/main/Test_CronJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-analytics-data
!pip install mysql-connector-python

  Using cached mysql_connector_python-9.0.0-cp310-cp310-manylinux_2_17_x86_64.whl.metadata (2.0 kB)
Using cached mysql_connector_python-9.0.0-cp310-cp310-manylinux_2_17_x86_64.whl (19.3 MB)


In [2]:
from google.analytics.data import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest
import mysql.connector

In [3]:
# สร้างไคลเอนต์เพื่อเชื่อมต่อกับ GA4
client = BetaAnalyticsDataClient()
client

In [20]:
# กำหนดการร้องขอข้อมูล
def run_report(property_id):
    request = RunReportRequest(
      property=f"properties/{property_id}",  # ใส่ Property ID ของ GA4
      dimensions=[{"name":"country"}],
      metrics=[{"name": "activeUsers"}],
      date_ranges=[{"start_date": "2023-01-01", "end_date": "today"}],
    )
    response = client.run_report(request)
    return response

In [21]:
property_id ='307754729'
property_id

'307754729'

In [22]:
try:
    report = run_report(property_id)
    # การทำงานเมื่อการดึงข้อมูลสำเร็จ
except Exception as e:
    print(f"An error occurred: {e}")

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7eecc211d300>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

An error occurred: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7eecad351060>)


In [23]:
for row in report.rows:
    print(f"Country: {row.dimension_values[0].value}, Active Users: {row.metric_values[0].value}")

NameError: name 'report' is not defined

In [ ]:
# เชื่อมต่อกับ MySQL
connection = mysql.connector.connect(
    host="159.138.255.180",
    user="root",
    password="Vcb168899@",
    database="vcb_ba2"
)
connection

In [ ]:
cursor = connection.cursor()
cursor

In [ ]:
with connection.cursor() as cursor:
    sql = "INSERT INTO active_users (country, active_users) VALUES (%s, %s)"
    for row in report.rows:
        cursor.execute(sql, (row.dimension_values[0].value, row.metric_values[0].value))

In [ ]:
connection.commit() # save การเปลี่ยนแปลง
connection.close() # ปิดการเชื่อมต่อ